<a href="https://colab.research.google.com/github/LucasPequenoSterzeck/Machine_Learning_LPS/blob/main/PyTorch_Jovian_ZeroToGANs/pytorch_lesson01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pytorch-lesson01

This notebook will be used foor practice in Lesson 1 PyTorch on Jovian

In [1]:
import torch

In [2]:
# Tensores são matrizes! podemos ter diversas dimensões, as mais comuns são:

# 1 Dimensão
t1 = torch.tensor([4.,2,1])
print(f'--> Aqui temos um tensor de 1 dimensão: {t1} |\nDimensões: {t1.shape}\n')

# 3 Dimensão
t2 = torch.tensor([[4.,2,1],[4.,2,1]])
print(f'--> Aqui temos um tensor de 3 dimensão: {t2} |\nDimensões: {t2.shape}\n')

# 3 Dimensão
t3 = torch.tensor([[[4.,2,1],[4.,2,1]],[[4.,2,1],[4.,2,1]]])
print(f'--> Aqui temos um tensor de 4 dimensão: {t3} |\nDimensões: {t3.shape}\n')

--> Aqui temos um tensor de 1 dimensão: tensor([4., 2., 1.]) |
Dimensões: torch.Size([3])

--> Aqui temos um tensor de 3 dimensão: tensor([[4., 2., 1.],
        [4., 2., 1.]]) |
Dimensões: torch.Size([2, 3])

--> Aqui temos um tensor de 4 dimensão: tensor([[[4., 2., 1.],
         [4., 2., 1.]],

        [[4., 2., 1.],
         [4., 2., 1.]]]) |
Dimensões: torch.Size([2, 2, 3])



Na criação de tensores temos opção de habilitar "requires_grad", essa função irá possibilitar o retrocesso dos valores via ".backward()" conforme demonstrado abaixo

In [3]:

# Criando tensores com opção "requires_grad"
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
print(f'x = {x}\nw = {w}\nb = {b}\n')

# operação aritmética:
y = w * x + b
print(f'y = {y}')
print(f'x = {x}\nw = {w}\nb = {b}\n')

# Ao chamar y.backward(), o PyTorch calcula automaticamente os gradientes de y em relação aos tensores...
# com requires_grad=True, ou seja, w e b. Os gradientes são então armazenados nos atributos grad dos respectivos tensores.
y.backward()

# Verificando gradientes
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)


x = 3.0
w = 4.0
b = 5.0

y = 17.0
x = 3.0
w = 4.0
b = 5.0

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


In [4]:
# Integração com Numpy:
import numpy as np

x = np.array([[[1, 2], [3, 4.]],[[1, 2], [3, 4.]],[[1, 2], [3, 4.]]])

# Convertendo de Numpy para Tensor Torch
y = torch.from_numpy(x)
y


tensor([[[1., 2.],
         [3., 4.]],

        [[1., 2.],
         [3., 4.]],

        [[1., 2.],
         [3., 4.]]], dtype=torch.float64)

Banco de dados que usaremos para criar lógicas:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

Iremos prever a produção de Maçãs(Apples) e Laranjas(Oranges) in toneladas (ton), considerando que cada fruta será uma regressão a parte (Já que não estamos trabalhando com regressão multinível), teremos:

> **yield_apple**  = w11 * temp + w12 * rainfall + w13 * humidity + b1

> **yield_orange** = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [5]:
# Criando banco de dados:

# Input (temp, rainfall, humidity) <- Valores presentes nas colunas 1 a 3
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges) <- Valores alvo que queremos prever
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')
# Convertendo tudo para Torch
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [6]:
# Agora que temos os dados, vamos configurar como o modelo irá mensurar o erro
# utilizaremos o Minimo Erro Quadrático ou MSE

def mse(t1, t2):
    # t1 são valores target/reais;
    # t2 são valores que o modelo encontrou como target/previsões
    # diff é a diferença de um para outro
    diff = t1 - t2
    # realizados diff * diff para elevar a diferença eliminando assim números negativos
    # o resultado será dívidido pela quantidade de termos presentes (diff.numel())
    return torch.sum(diff * diff) / diff.numel()


O calculo do gradiante será realizado e ajustado de acordo com o erro que encontrarmos, por isso que a etapa acima é tão importante para o modelo.

Abaixo exemplo gráfico, nosso objetivo com essa imagem é sempre chegar o ponto mais "baixo" possível do gradiante, pois é lá onde o erro é o menor possível.

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

# Linear regression model from scratch

In [7]:
# W simbolizará relações que serão multiplicadas pelas colunas pra que encontremos os valores procurados;
# B são a influência aplicada que irá nos gerar os valores de predição

# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

# torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

tensor([[-1.2310, -0.0236, -0.3587],
        [-0.3754, -0.8476,  0.3598]], requires_grad=True)
tensor([-0.9626,  2.1724], requires_grad=True)


In [8]:
# Nosso modelo será elaborado da seguinte forma:
def model(x):
    return x @ w.t() + b

O que o modelo irá realizar:

![matrix-mult](https://i.imgur.com/WGXLFvA.png)


Na primeira Linha de X, modelo irá fazer 73 x w11, 67 x W12 e 43 x w13 + b1.

Da mesma forma será feito com a segunda coluna de W:
  73 x w21
  67 x w22
  43 x w23
  '+ b2


In [ ]:
# Valores que nosso modelo chegou atualmente
print('Valores que nosso modelo preveu para cada caso:')
preds = model(inputs)
print(preds)

# Alvo que é o valor referência
print('\nValores que precisamos chegar, para cada casoi:')
print(targets)

Valores que nosso modelo preveu para cada caso:
tensor([[  14.2309,  -63.7383],
        [  25.2311,  -74.7806],
        [  25.3634,  -57.5764],
        [  -0.5075, -110.1792],
        [  36.7349,  -42.1841]], grad_fn=<AddBackward0>)

Valores que precisamos chegar, para cada casoi:
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Perceba que em alguns casos nosso modelo preveu valores negativos, o que não é um bom indício, mas tudo bem, estamos falando da primeira versão dele, e consideremos que ele foi inicializado com pesos aleatórios, o que precisamos agora é **computar o erro** e conseguir evoluir apartir desse ponto, para isso precisamos aplicar uma metrica aos erros, que será **mse** já apresentado.

In [ ]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [ ]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(15126.6465, grad_fn=<DivBackward0>)


In [ ]:
print('Valores W antes de realizar backward:')
print(w)
print(w.grad)

print('\nValores W após backward:')
loss.backward()
print(w)
print(w.grad)

Valores W antes de realizar backward:
tensor([[-0.3029,  0.0696,  0.7138],
        [-1.2952,  0.2986,  0.2857]], requires_grad=True)
None

Valores W após backward:
tensor([[-0.3029,  0.0696,  0.7138],
        [-1.2952,  0.2986,  0.2857]], requires_grad=True)
tensor([[ -4627.7129,  -5516.5557,  -3253.5088],
        [-13694.6133, -14353.7559,  -8956.5312]])


## Ajustando os pesos para buscar reduzir o erro

A perca/erro é uma função quadrática dos pesos, então nosso objetivo é encontrar o conjunto de pesos onde o erro é menor.

**Se o elemento que encontrarmos for POSITIVO, então:**
<br>
> + AUMENTAR os pesos do elemento farão que o erro aumente;
> + DIMINUIR os pesos do elemento farão que o erro diminua;

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)
------------------------

**Se o elemento que encontrarmos for NEGATIVO, então:**
<br>
> + AUMENTAR os pesos do elemento farão que o erro diminua;
> + DIMINUIR os pesos do elemento farão que o erro aumente;

![negative=gradient](https://i.imgur.com/dvG2fxU.png)
------------------------

Agora o único ponto que devemos tomar cuidado é o quanto (Qual a intensidade) que iremos acatar essa direção, se dermos passos muito distantes podemos jogar o modelo "para longe", por conta disso sempre iniciamos a adotar pequenas alterações sempre na direção que diminua nosso erro.

In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5


In [ ]:
# Agora vamos verificar para ver se o erro está menor:
loss = mse(preds, targets)
print(loss)

tensor(15126.6465, grad_fn=<DivBackward0>)


In [ ]:
# Agora vamos refazer o processo
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor([[ 22.7048, -40.2713],
        [ 36.3797, -43.9534],
        [ 38.6693, -21.2317],
        [  7.7892, -86.7231],
        [ 47.5020, -12.6840]], grad_fn=<AddBackward0>)
tensor(10244.1299, grad_fn=<DivBackward0>)


In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -3751.7290,  -4572.1133,  -2671.4485],
        [-11272.9346, -11754.5742,  -7351.9507]])
tensor([ -45.5910, -132.9727])


In [ ]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()
print(w)
print(b)

tensor([[-0.2191,  0.1705,  0.7731],
        [-1.0455,  0.5597,  0.4488]], requires_grad=True)
tensor([ 0.9839, -1.4819], requires_grad=True)


In [ ]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(6953.2549, grad_fn=<DivBackward0>)


Para agilizar vamos criar um loop e repetir o processo 1000 vezes:

In [ ]:
# Train for 1000 epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
# Verificando modelo após 1000 epochs
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.5134, grad_fn=<DivBackward0>)


# Linear regression using PyTorch built-ins